In [44]:
# 1. скачать набор данных маркетинговых кампаний отсюда https://www.kaggle.com/davinwijaya/customer-retention
# 2. там поле conversion - это целевая переменная, а offer - коммуникация. 
# Переименовать поля (conversion -> target, offer -> treatment) и привести поле treatment к бинарному виду 
# (1 или 0, т.е было какое-то предложение или нет) - значение No Offer означает отсутствие коммуникации, 
# а все остальные - наличие.
# 3. сделать разбиение набора данных не тренировочную и тестовую выборки
# 4. сделать feature engineering на ваше усмотрение (допускается свобода выбора методов)
# 5. провести uplift-моделирование 3 способами: одна модель с признаком коммуникации (S learner), 
# модель с трансформацией таргета (трансформация классов п. 2. 1) и вариант с двумя независимыми моделями
# 6. в конце вывести единую таблицу сравнения метрик uplift@10%, uplift@20% этих 3 моделей
# 7. построить модель UpliftTreeClassifier и попытаться описать словами полученное дерево
# 8. (опционально) для модели S learner (модель с дополнительным признаком коммуникации) построить зависимость таргета 
# (конверсии - поле conversion) от значения uplift: 
#     1) сделать прогноз и получить uplift для тестовой выборки 
#     2) отсортировать тестовую выборку по uplift по убыванию 
#     3) разбить на децили (pandas qcut вам в помощь) 
#     4) для каждого дециля посчитать среднюю conversion
# 9. (опционально) построить модель UpliftRandomForestClassifier и попытаться описать словами полученное дерево

In [45]:
%matplotlib inline

import numpy as np
import pandas as pd; pd.set_option('display.max_columns', None)
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import FeatureUnion
# !pip install scikit-uplift==0.2.0
# pip install catboost
from sklift.metrics import uplift_at_k
from sklift.viz import plot_uplift_preds
from sklift.models import SoloModel


# Чтение данных
df = pd.read_csv('data.csv')

In [ ]:
from sklearn.base import TransformerMixin #gives fit_transform method for free
class MyLabelBinarizer(TransformerMixin):
    def __init__(self, *args, **kwargs):
        self.encoder = LabelBinarizer(*args, **kwargs)
    def fit(self, x, y=0):
        self.encoder.fit(x)
        return self
    def transform(self, x, y=0):
        return self.encoder.transform(x)

In [107]:
TransformerMixin?

In [108]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key, *args, **kwargs):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]

In [47]:
df.head(3)

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,offer,conversion
0,10,142.44,1,0,Surburban,0,Phone,Buy One Get One,0
1,6,329.08,1,1,Rural,1,Web,No Offer,0
2,7,180.65,0,1,Surburban,1,Web,Buy One Get One,0


In [48]:
df.rename(columns={'conversion': 'target', 'offer': 'treatment'}, inplace=True)
df.head(3)

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,treatment,target
0,10,142.44,1,0,Surburban,0,Phone,Buy One Get One,0
1,6,329.08,1,1,Rural,1,Web,No Offer,0
2,7,180.65,0,1,Surburban,1,Web,Buy One Get One,0


In [49]:
df['treatment'].value_counts()

Buy One Get One    21387
Discount           21307
No Offer           21306
Name: treatment, dtype: int64

In [50]:
dict_ = {'Buy One Get One':1, 'Discount':1, 'No Offer':0}
df = df.replace({'treatment': dict_})

In [51]:
df['treatment'].value_counts()

1    42694
0    21306
Name: treatment, dtype: int64

In [89]:
features = df.columns[:-2]
target = df.columns[-1:]
treatment = df.columns[-2:-1]

X_train, X_test, treat_train, y_train, y_test, treat_test = train_test_split(df[features], df[target], df[treatment], test_size=0.3, random_state=42)

In [90]:
X_train.head(3)

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel
9656,7,434.35,1,0,Urban,1,Web
63037,1,376.59,1,0,Surburban,0,Multichannel
31405,3,140.34,0,1,Urban,1,Phone


In [91]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44800 entries, 9656 to 56422
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   recency        44800 non-null  int64  
 1   history        44800 non-null  float64
 2   used_discount  44800 non-null  int64  
 3   used_bogo      44800 non-null  int64  
 4   zip_code       44800 non-null  object 
 5   is_referral    44800 non-null  int64  
 6   channel        44800 non-null  object 
dtypes: float64(1), int64(4), object(2)
memory usage: 2.7+ MB


In [92]:
# К полям:
# zip_code, channel применим OHE-кодирование
# history - standardScaler
# recency, used_discount, used_bogo, is_referral, treatment - оставим как есть

continuos_cols = ['history']
cat_cols = ['zip_code', 'channel']
base_cols = ['recency', 'used_discount', 'used_bogo', 'is_referral']

continuos_transformers = []
cat_transformers = []
base_transformers = []

for cont_col in continuos_cols:
    transfomer =  Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('standard', StandardScaler())
            ])
    continuos_transformers.append((cont_col, transfomer))
    
for cat_col in cat_cols:
    cat_transformer = Pipeline([
                ('selector', ColumnSelector(key=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    cat_transformers.append((cat_col, cat_transformer))
    
for base_col in base_cols:
    base_transformer = Pipeline([
                ('selector', NumberSelector(key=base_col))
            ])
    base_transformers.append((base_col, base_transformer))

In [93]:
# объединим все трансформеры с помощью FeatureUnion
feats = FeatureUnion(continuos_transformers + cat_transformers + base_transformers)
feature_processing = Pipeline([('feats', feats)])

feature_processing.fit_transform(X_train)

array([[ 0.75403071,  0.        ,  0.        , ...,  1.        ,
         0.        ,  1.        ],
       [ 0.52773036,  0.        ,  1.        , ...,  1.        ,
         0.        ,  0.        ],
       [-0.39788353,  0.        ,  0.        , ...,  0.        ,
         1.        ,  1.        ],
       ...,
       [ 0.9819766 ,  1.        ,  0.        , ...,  1.        ,
         1.        ,  1.        ],
       [-0.66453869,  0.        ,  0.        , ...,  1.        ,
         0.        ,  1.        ],
       [ 2.13769124,  0.        ,  0.        , ...,  1.        ,
         1.        ,  1.        ]])

#### 1.1 Одна модель с признаком коммуникации
Модель обучается одновременно на двух группах, при этом бинарный флаг коммуникации выступает в качестве дополнительного признака. Каждый объект из тестовой выборки скорим дважды: с флагом коммуникации равным 1 и равным 0. Вычитая вероятности по каждому наблюдению, получим искомы uplift.

In [109]:
# добавим модель 
uplift_sm = Pipeline([
    ('features', feats),
    ('uplift_model', SoloModel(CatBoostClassifier(iterations=20, thread_count=3, random_state=42, silent=True))),
])

# обучим пайплайн на тренировочном датасете
uplift_sm.fit(X_train, y_train, treat_train)
y_preds_uplift_sm = uplift_sm.predict(X_test)

sm_score = uplift_at_k(y_true=y_test, uplift=y_preds_uplift_sm, treatment=treat_test, strategy='by_group', k=0.1)
print(f'uplift@10%: {sm_score:.4f}')

table_comp.loc[0] = ['SoloModel', sm_score]

# Получим условные вероятности выполнения целевого действия при взаимодействии для каждого объекта
sm_trmnt_preds = sm.trmnt_preds_
# И условные вероятности выполнения целевого действия без взаимодействия для каждого объекта
sm_ctrl_preds = sm.ctrl_preds_

# Отрисуем распределения вероятностей и их разность (uplift)
plot_uplift_preds(trmnt_preds=sm_trmnt_preds, ctrl_preds=sm_ctrl_preds);

TypeError: fit() takes from 2 to 3 positional arguments but 4 were given

In [97]:
# Сводная таблица по моделям
table_comp = pd.DataFrame({'Model': [], 
                           'uplift@10%': []})